## Exploration of large scientific dataset with OpenVisus and ITKwidgets

### OpenViSUS: install package

To execute this jupyter notebook have to install: OpenViSUS
```
python -m pip install OpenVisus

python -m OpenVisus configure
```

Note: ignore errors during the "configure" process

Alternatively, see below on how to install the packages directly from the jupyter notebook

### ITKwidgets: install package

You can install itkwidgets from the anaconda-navigator (environment tab, search for "itkwidgets" in "not installed packages).

Alternatively you can install it from an Anaconda console/prompt with:

`conda install itkwidgets`

### OpenViSUS: read from a remote dataset

In [1]:
%matplotlib notebook

import os,sys

# Here are commands to install a package (OpenVisus) directly from a jupyter notebook
# after you install those once you can comment those comment
!{sys.executable} -m pip install OpenVisus

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *

import OpenVisus as ov

# Enable I/O component of OpenVisus
ov.DbModule.attach()

Starting OpenVisus c:\python38\lib\site-packages\OpenVisus\__init__.py 3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)] sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0) ...


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
# function to read data from a remote dataset
# optional parameters: timestep, field (variable in the dataset), logic_box (bounding box of the query), resolution

# Note: the resolution value could sometime fetch a dataset with the wrong aspect ratio, 
# this because in the IDX format we double the size at each resolution on only one of the axis at a time

# function to plot the image data with matplotlib
# optional parameters: colormap, existing plot to reuse (for more interactivity)
def showData(data, cmap=None, plot=None):
    if(plot==None or cmap!=None):
        fig=plt.figure(figsize = (7,5))
        plot = plt.imshow(data, origin='lower', cmap=cmap)
        plt.show()
        return plot
    else:
        plot.set_data(data)
        plt.show()
        return plot

### Navigate time and resolution

In [3]:
# select a remote dataset (satellite imagery from NASA)
# this doesn't actually fetch any data, only metadata
dataset=ov.LoadDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=BlueMarble")

# what is the size of this dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0], [86400, 43200])

In [4]:
# what is the maximum resolution ? 
# NOTE: don't use large values of the resolution for large query (you will be fetching too much data)
dataset.getMaxResolution()

33

In [5]:
# what timesteps are defined
dataset.getTimesteps().toString()

'<DatasetTimesteps>\n\t<timestep from="0" to="11" step="1" />\n</DatasetTimesteps>'

In [6]:
showData(dataset.read(time=0, max_resolution=21))

<IPython.core.display.Javascript object>

In [7]:
# create a plot for our data
myplot = showData(dataset.read(time=0, max_resolution=21))

# reuse the plot with an interact for varying time and resolution values
interact(
    lambda time,resolution: showData(dataset.read(time=time,max_resolution=resolution), plot=myplot),
    time=widgets.IntSlider(value=0,min=0,max=11,step=1), 
    resolution=widgets.IntSlider(value=9,min=1,max=dataset.getMaxResolution(),step=2))

<IPython.core.display.Javascript object>

interactive(children=(IntSlider(value=0, description='time', max=11), IntSlider(value=9, description='resoluti…

<function __main__.<lambda>(time, resolution)>

### Interactive analysis

In [8]:
# Open an aerial dataset from the National Ecology Observatory Network 
dataset=ov.LoadDataset("https://molniya.sci.utah.edu/mod_visus?dataset=neon_redb")
dataset.getLogicBox()

([0, 0], [80000, 100000])

In [9]:
data = dataset.read(max_resolution=22)
showData(data)

<IPython.core.display.Javascript object>

In [10]:
# what's the name of the field we are looking at?
dataset.getField().name

'DATA'

In [11]:
# what's the size of the data fetched? (note: there are three channels, RGB)
data.shape

(1563, 1250, 3)

In [12]:
# make a "grey scale" version of the data
# from Matlab "rgb2gray" 0.2989 * R + 0.5870 * G + 0.1140 * B (standard ITU-R BT.601-7)

R,G,B=(0.2989*data[:,:,0], 0.5870*data[:,:,1], 0.1140*data[:,:,2])
grey_data=R+G+B

In [13]:
# show data using a grey scale colormap
showData(grey_data, cmap=plt.get_cmap("Greys"))

<IPython.core.display.Javascript object>

In [14]:
# make a threshold function to show which "pixel" is above a certain values
def threshold(data, t):
    return data > t

In [15]:
showData(threshold(grey_data, 150))

<IPython.core.display.Javascript object>

In [16]:
# make the threshold exploration interactive
myplot = showData(threshold(grey_data,t=150))

interact(
    lambda thr: showData(threshold(grey_data,t=thr), plot=myplot),
    thr=widgets.IntSlider(value=np.mean(grey_data),min=np.min(grey_data),max=np.max(grey_data),step=1))

<IPython.core.display.Javascript object>

interactive(children=(IntSlider(value=96, description='thr', max=254), Output()), _dom_classes=('widget-intera…

<function __main__.<lambda>(thr)>

### Working with 3D datasets

In [17]:
dataset=ov.LoadDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1")
# how big is the dataset ?
dataset.getLogicBox()

([0, 0, 0], [2048, 2048, 2048])

In [18]:
# make a query to fetch a slice of this 3D dataset (in the middle of the 3rd axis)
data=dataset.read(x=[0,2048],y=[0,2048],z=[1024,1025],max_resolution=21)
showData(data[0,:,:])

<IPython.core.display.Javascript object>

### Use ITK widgets

In [20]:
# import itk libraries and the "view" function

!{sys.executable} -m pip install itk itkwidgets


import itk
from itkwidgets import view
import itkwidgets

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

ipympl 0.6.2 requires ipywidgets>=7.6.0, but you'll have ipywidgets 7.5.1 which is incompatible.
You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


  Using cached itkwidgets-0.32.0-py2.py3-none-any.whl (3.4 MB)
  Using cached colorcet-2.0.2-py2.py3-none-any.whl (1.6 MB)
  Using cached pyct-0.4.8-py2.py3-none-any.whl (15 kB)


In [21]:
# use itkwidgets to explore and visualize 2D data
view(data)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [22]:
# read the entire dataset at a certain resolution
data=dataset.read(max_resolution=24)

In [23]:
# show it in 3D
view(data, select_roi=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [24]:
# loading a large 2D microscopy dataset showing the retina of a rabbit
dataset=ov.LoadDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit")
# how big is the dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0], [131072, 131072])

In [26]:
# get default field name
dataset.getField().name

'EM'

In [27]:
data=dataset.read(max_resolution=22)
view(data)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC2; pr…

In [28]:
# open a 3D version of this dataset
dataset=ov.LoadDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit3d")
# how big is the dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0, 0], [131072, 131072, 341])

In [ ]:
# almost 6TB size, can we still visualize it on this browser?
131073*131073*342/(1024*1024*1024)

In [29]:
data=dataset.read(max_resolution=21)
view(data)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

### Voxel spacing

Microscopy data or CT scan often have different resolution along the axis, in order to make a realistic visualization we need to use correct voxel spacing

In [30]:
# to edit the voxel spacing we need to transform our numpy array into an itk data structure
itk_array = itk.image_from_array(data)
# read the current spacing (uniform)
itk_array.GetSpacing()

itkVectorD3 ([1, 1, 1])

In [31]:
# set a new spacing
itk_array.SetSpacing([1.0, 1.0, 0.1])
view(itk_array)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

In [32]:
# we open a metallic foam dataset
dataset=ov.LoadDataset("https://molniya.sci.utah.edu/mod_visus?dataset=foam")
# how big is the dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0, 0], [1055, 1024, 1024])

In [33]:
# Visualize and explore the dataset 
# How can we evaluate the density of material?

data=dataset.read(max_resolution=18)
view(data, select_roi=True)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUS3; pr…

In [34]:
# Evaluate the density of material 
count = (data > 23000).sum()
density = count/(data.shape[0]*data.shape[1]*data.shape[2])
density


0.04525479403409091

In [39]:
def density_res(res):
    data=dataset.read(max_resolution=res)
    count = (data > 23000).sum()
    density = count/(data.shape[0]*data.shape[1]*data.shape[2])
    return density

In [40]:
density_res(21)

0.044757265033143936

In [ ]:
density_res(20)

In [ ]:
density_res(18)

In [ ]:
density_res(17)

In [ ]:
density_res(15)

In [ ]:
density_res(10)

In [41]:
density_res(23)

0.04476350726503314

### Run client OpenVisus Viewer 

#### Note: the following does not work in a remote environment (e.g.,  Binder or JupyterHub), you can try the following code downloading this jupyter notebook and running it locally on your machine (after installing the dependencies OpenVisus and ITKwidgets, see beginning of this page)

In [46]:
from OpenVisus   import *
from VisusGuiPy	 import *

In [47]:
SetCommandLine("__main__")
GuiModule.createApplication()
AppKitModule.attach()  
viewer=Viewer()
# select dataset to open with the viewer
viewer.open("https://molniya.sci.utah.edu/mod_visus?dataset=foam") 
GuiModule.execApplication()

AttributeError: type object 'GuiModule' has no attribute 'createApplication'